In [ ]:
import os, re, matplotlib, pandas, collections, importlib, sys
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import __init__
sys.path = __init__.add_paths(sys.path)

from importlib import reload
import sameRiver
from sameRiver import *
import pandas as pd




In [ ]:
beds = '/Users/dfporter/pma/dataAndScripts/clip/miseq/Runs/180815/beds/'
scheme = '/Users/dfporter/pma/dataAndScripts/clip/miseq/Runs/180815/scheme.xlsx'

top_dir = '/Users/dfporter/pma/dataAndScripts/clip/miseq/all/'
importlib.reload(sameRiver.exp)
importlib.reload(sameRiver.scheme)
ex = sameRiver.exp.exp(name='all', file_paths={
    'scheme': top_dir + '/scheme.xls',
    'beds': top_dir + '/beds/',
    'wigs': top_dir + '/beds/read_start/',
    'fastq': top_dir + '/fastq/',
    'sams': top_dir + '/sams/',
    'STAR index': '/labs/khavari/dfporter/genome/GRCh38.gencode.29/star_index',
    'STAR': 'STAR',
    'STAR repeats index': '/labs/khavari/dfporter/genome/repeats/star_repeats/',
    })

ex.read_scheme()
print(os.getcwd())

if os.path.exists('/Users/dfporter/pma/dataAndScripts/clip/miseq/all/line_numbers.xlsx'):
    df = pandas.read_excel('/Users/dfporter/pma/dataAndScripts/clip/miseq/all/line_numbers.xlsx')
else:
    print("Run ex.line_numbers() to generate file.")

for col in ['Gene', 'Replicate']:#, 'Percent XL']:
    df[col] = [ex.scheme.long_basename_to_info.get(x, '')[col] for x in df.basename]
#df['Percent XL'] = ['{:.3%}'.format(x) for x in df['Percent XL']]
df['Total read pairs'] = [x/4 for x in df['r1r2_split line number']]
df['Read pairs with insert'] = [x/4 for x in df['r1r2_clipped line number']]
df['Mapped, unique reads'] = df['bed line number']
df['% adapter dimer'] = ['{:.2%}'.format((tot-inst)/tot) for tot,inst in zip(
    df['Total read pairs'], df['Read pairs with insert'])]
df['% mapped, unique (of reads with insert)'] = [
    '{:.2%}'.format(mapped/inst) for mapped, inst in zip(
    df['Mapped, unique reads'], df['Read pairs with insert'])
]
df = df[['Gene', 'Replicate', 'Total read pairs', 'Read pairs with insert',
        'Mapped, unique reads', '% adapter dimer', '% mapped, unique (of reads with insert)']]
df['Gene/rep'] = ['{} {}'.format(x, y) for x, y in zip(df.Gene, df.index)]
df.sort_values(by='Mapped, unique reads', inplace=True)

fig = plt.figure()
sns.set_style('ticks')
#sns.set_style('whitegrid')
sns.barplot(y='Gene/rep', x='Mapped, unique reads', log=True, data=df, color='k')
#plt.plot([1E5,1E5], [0, len(df.index)], 'k')
fig.set_figheight(7)
plt.show()
plt.clf()
print(df)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import six

#https://stackoverflow.com/questions/26678467/export-a-pandas-dataframe-as-a-table-image

def render_mpl_table(data, col_width=3.0, row_height=0.625, font_size=14,
                     header_color='#40466e', row_colors=['#f1f1f2', 'w'], edge_color='w',
                     bbox=[0, 0, 1, 1], header_columns=0,
                     ax=None, **kwargs):
    if ax is None:
        size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array([col_width, row_height])
        fig, ax = plt.subplots(figsize=size)
        ax.axis('off')

    mpl_table = ax.table(cellText=data.values, bbox=bbox, colLabels=data.columns, **kwargs)

    mpl_table.auto_set_font_size(False)
    mpl_table.set_fontsize(font_size)

    for k, cell in  six.iteritems(mpl_table._cells):
        cell.set_edgecolor(edge_color)
        if k[0] == 0 or k[1] < header_columns:
            cell.set_text_props(weight='bold', color='w')
            cell.set_facecolor(header_color)
        else:
            cell.set_facecolor(row_colors[k[0]%len(row_colors) ])
    return ax


In [ ]:
_scheme = sameRiver.scheme.scheme('/Users/dfporter/pma/dataAndScripts/clip/miseq/meta/scheme.xls')

df = _scheme.scheme_df
df['ID'] = ['_'.join(str(x) for x in [gene, l5, l3]) for gene, l5, l3 in zip(
    df.Gene, df.P6_BC, df.P3_BC)]

df = df[[('STD' not in _id) for _id in df.ID]]
df = df[[('nknown' not in _id) for _id in df.ID]]
df = df[[('100' not in _id) for _id in df.ID]]
df = df[[('PCBP1' not in _id) for _id in df.ID]]
df = df[[('SF3B1' not in _id) for _id in df.ID]]
df = df[[('YBX' not in _id) for _id in df.ID]]

for col in ['Reads', 'Mapped reads']:
    df[col] = np.nan_to_num(df[col].tolist())
df.sort_values(by='Gene', inplace=True)
df = df[['Gene', 'ID', 'Reads', 'Mapped reads']]
df['Reads'] = df.apply(lambda x: "{:,}".format(int(x['Reads'])), axis=1)
df['Mapped reads'] = df.apply(lambda x: "{:,}".format(int(x['Mapped reads'])), axis=1)

#??df.style.format({'Reads': "{:.2%}", 'Mapped reads': "{:,}"})
print(df)
ax = render_mpl_table(
    df[['Gene', 'Reads', 'Mapped reads']].copy(), 
    header_columns=0, col_width=1., font_size=5, row_height=0.2)

df['Reads'] = [int(x.replace(',', '')) for x in df['Reads']]
df['Mapped reads'] = [int(x.replace(',', '')) for x in df['Mapped reads']]

plt.savefig('/Users/dfporter/pma/dataAndScripts/clip/figs/table_of_read_numbers.pdf')
plt.clf()
plt.close()
print(df.head())

sns.set_style('whitegrid')
fig = plt.figure()

_kwargs = {'palette': sns.cubehelix_palette(2*len(set(df['ID'])), start=0, rot=-0.2)}

df['Reads (log 10)'] = np.log10(df['Reads'])
sns.barplot(data=df, x='Reads (log 10)', y='ID', **_kwargs)
fig.set_figheight(20)
plt.show()
plt.clf()

fig = plt.figure()

df['Mapped reads (log 10)'] = np.log10(df['Mapped reads'])
df.sort_values(by='Mapped reads', inplace=True, ascending=False)
sns.set_style('whitegrid')
#sns.set(font_scale=0.7)
sns.barplot(data=df, x='Mapped reads (log 10)', y='ID', **_kwargs)

fig.set_figheight(12)
plt.savefig('/Users/dfporter/pma/dataAndScripts/clip/figs/mapped_reads_log10_barplot.pdf')
plt.show()
plt.clf()

